In [1]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#To support GPU
!pip install opencv-python==4.4.0.46

     |████████████████████████████████| 49.5MB 77kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [3]:
import os
import cv2
import numpy as np

# for showing images in the cell outputs (Jupyter Notebooks / Google Colab)
from IPython.display import display
from ipywidgets import Image

BASE_DIR = '/content/gdrive/My Drive/mircv2021'
DEEP_PROTO = BASE_DIR + '/data/caffe/train_val.prototxt'
DEEP_MODEL = BASE_DIR + '/data/caffe/bvlc_reference_caffenet.caffemodel'
SRC_FOLDER = BASE_DIR + '/data/coco_img'

DEEP_LAYER = 'relu7'
SIZE = (227, 227)
MEAN_VALUES = (104, 117, 123)  # BGR

K = 10

print("Environment variables Done")

Environment variables Done


In [13]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values
        #TODO
        #Load Caffe Weights
        self.net = cv2.dnn.readNetFromCaffe(net_proto_path, trained_model_path)
        
        # to enable GPU (this won't work on Colab without recompiling opencv)
        #self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        #self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    def extract(self, img_file, layer, normalize=False):
        #TODO
        #Load img_file and extract fetures calling DNN 
        img = cv2.imread(img_file)
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)
        features = self.net.forward(layer).flatten()

        #normalize features if normalize = True
        if normalize == True:
          features = features/np.linalg.norm(features)
        #return the features
        return features
    
# it creates an instance of the DNNExtractor class
dnn = DNNExtractor(DEEP_PROTO, DEEP_MODEL, SIZE, MEAN_VALUES)
print("Done DNN")

Done DNN


In [16]:
def extract_features(img_folder):
  
    #TODO
    #get the list of image filenames
    filenames = []

    descriptors = []
    #TODO
    #cycle the img_folder directory to get the image files
    
    filenames = os.listdir(img_folder)

    for filename in filenames:
      if filename.endswith('.jpg'):
        #extract the image features
        filepath = os.path.join(img_folder, filename)
        #add the features to the descriptors list
        features = dnn.extract(filepath, DEEP_LAYER, normalize=True)

        descriptors.append(features)
    
    #convert descriptors and filenames as Numpy arrays
    return descriptors, filenames

descs, ids = extract_features(SRC_FOLDER)
print("Extract_features Done")

Extract_features Done


In [27]:
def search(queryF, k):
    #evaluate the dot products between descs and queryF
    dots = np.dot(descs, queryF)

    #To sort the results create a zip between dot products and ids
    results = zip(dots, ids)

    #then call the sorted function to sort them
    results = sorted(results, reverse=True)

    #return the k best results
    return results[:k]

def display_results(results):
    for score, filename in results:
        print('{} - {:.3f}'.format(filename, score))  # :.3f = 3-decimal float

        filepath = os.path.join(SRC_FOLDER, filename)
        image = Image.from_file(filepath, width=300, height=400)
        display(image)

# Bike: 000000321557
# Cat: 000000271728
# Person: 000000222118
# Car: 000000196759
# Bird: 000000489046
# Church: 000000484760
# Giraffe: 000000480842

img_query = os.path.join(SRC_FOLDER, "000000480842.jpg")
query_features = dnn.extract(img_query, DEEP_LAYER, True)

res = search(query_features, K)
print(res)
display_results(res)

[(0.99999994, '000000480842.jpg'), (0.6092424, '000000492284.jpg'), (0.57328284, '000000205282.jpg'), (0.56985784, '000000333069.jpg'), (0.5452732, '000000079188.jpg'), (0.5293269, '000000336232.jpg'), (0.5172737, '000000150726.jpg'), (0.5170654, '000000229753.jpg'), (0.509569, '000000022969.jpg'), (0.46589303, '000000216739.jpg')]
000000480842.jpg - 1.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\…

000000492284.jpg - 0.609


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\…

000000205282.jpg - 0.573


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01^\x01^\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

000000022969.jpg - 0.510


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000216739.jpg - 0.466


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\…

In [34]:
#Optional task

#TODO
# binary descriptors. For each value set 1 if value > 0, 0 otherwise
descs_bin = None
descs_bin = np.where(query_features == 1,1,1)
print(descs_bin)


def search_bin(queryF_bin, k):
    #evaluate Hamming distance between descs_bin and queryF_bin
    hamming = []
    for desc in descs_bin:
      ham = np.sum(np.bitwise_xor(desc, queryF_bin))
      hamming.append(ham)

    #then call the sorted function to sort them
    res = zip(hamming, ids)
    res = sorted(res, reverse=True)
    #return the k best results

    return res[:k]

[1 1 1 ... 1 1 1]


In [35]:
#Optional task

#TODO
# binary query. For each value set 1 if value > 0, 0 otherwise
queryF_bin = None
queryF_bin = np.where(query_features == 0,0,1)
print(queryF_bin)

res = search_bin(queryF_bin, K)
print(res)
display_results(res)


[1 1 1 ... 1 1 1]
[(0, '000000507667.jpg'), (0, '000000507473.jpg'), (0, '000000507081.jpg'), (0, '000000507042.jpg'), (0, '000000507037.jpg'), (0, '000000506454.jpg'), (0, '000000506310.jpg'), (0, '000000505942.jpg'), (0, '000000505638.jpg'), (0, '000000505573.jpg')]
000000507667.jpg - 0.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000507473.jpg - 0.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\…

000000507081.jpg - 0.000


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000505638.jpg - 0.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01+\x01+\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\…

000000505573.jpg - 0.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x1cmICC_PROFILE\x00\x01\x…